In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import re
import fasttext

In [17]:
fasttext_model = fasttext.load_model('D:/Chipsal/Embeddings/devanagari_fasttext_cbow.bin')

In [18]:
input_size = fasttext_model.get_dimension()
hidden_size = 256
num_layers = 3
num_classes = 5
num_epochs = 10
learning_rate = 0.001

In [19]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, lstm_output):
        scores = self.Va(torch.tanh(self.Wa(lstm_output) + self.Ua(lstm_output)))
        attention_weights = torch.softmax(scores, dim=1)
        
        context_vector = torch.bmm(attention_weights.permute(0, 2, 1), lstm_output)
        return context_vector.squeeze(1), attention_weights.squeeze(2)

class BidirectionalLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BidirectionalLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True, bidirectional=True)
        self.attention = Attention(hidden_size * 2)  
        self.fc = nn.Linear(hidden_size * 2, num_classes)  

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device) 
        
        out, _ = self.lstm(x, (h0, c0)) 
        

        context_vector, attention_weights = self.attention(out)
     
        out = self.fc(context_vector)
        return out

In [20]:
class DevanagariDataset(Dataset):
    def __init__(self, texts, max_length=50):
        self.texts = texts.tolist()  
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        words = self.texts[idx].split()  
        
        embeddings = [torch.tensor(fasttext_model.get_word_vector(word)) for word in words]
        
        if len(embeddings) > self.max_length:
            embeddings = embeddings[:self.max_length]  
        else:
            padding = [torch.zeros(fasttext_model.get_dimension())] * (self.max_length - len(embeddings))  
            embeddings.extend(padding)
        
        text_tensor = torch.stack(embeddings)

        return text_tensor

In [21]:
test_df = pd.read_csv("D:/Chipsal/ST1/datasets/test.csv")  
test_dataset = DevanagariDataset(test_df["text"])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [22]:
model = BidirectionalLSTMModel(input_size, hidden_size, num_layers, num_classes)
model.load_state_dict(torch.load("D:/Chipsal/ST1/lstm_attention_model.pth"))
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BidirectionalLSTMModel(
  (lstm): LSTM(100, 256, num_layers=3, batch_first=True, bidirectional=True)
  (attention): Attention(
    (Wa): Linear(in_features=512, out_features=512, bias=True)
    (Ua): Linear(in_features=512, out_features=512, bias=True)
    (Va): Linear(in_features=512, out_features=1, bias=True)
  )
  (fc): Linear(in_features=512, out_features=5, bias=True)
)

In [24]:
predictions = []
with torch.no_grad():
    for texts in test_loader:
        texts = texts.to(device)
        outputs = model(texts)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())

# Save predictions to a DataFrame
test_df['prediction'] = predictions

In [25]:
test_df

,index,text,prediction
0,10004,त्यो आँ गरेको अन्वार मा झिँगा पस्न सक्छ है? हि...,0
1,10005,निर्वाचन परिणाम ले बल्ल बुद्धि आयो?,0
2,10007,पोखराको मेयर उम्मेदवारी दिन कृष्ण थापाले दिए स...,0
3,10011,दलित महिला सदस्यमा उम्मेदवारी परेन,0
4,10018,पार्टी निर्णय बिपरीत उम्मेद्वारी दिने दिनेलाई ...,0
...,...,...,...
11229,84860,राज्य सरकारला पाच वर्षे होत आले आहेत मात्र हे ...,1
11230,84862,"महाविकास आघाडी बाबुभैया, “हेराफेरी”तो है!राज्य...",1
11231,84865,एकात्मिक बालविकास कार्यक्रमाअंतर्गत कोल्हापूर ...,1
11232,84876,अत्यल्प भाव मिळालेल्या शेतकऱ्यांना किमान कोटी ...,1


In [26]:
test_df.drop(columns=['text'], inplace=True)

In [27]:
test_df = test_df.sort_values(by="index")
test_df.head()

,index,prediction
0,10004,0
1,10005,0
2,10007,0
3,10011,0
4,10018,0


In [28]:
output_file_path = 'output.txt'

# Write the DataFrame to a text file in the desired format
with open(output_file_path, 'w') as f:
    for index, row in test_df.iterrows():
        f.write(f'{{"index": {row["index"]}, "prediction": {row["prediction"]}}}\n')